In [3]:
import pandas as pd
import glob
from pathlib import Path
import numpy as np
from numpy.random import default_rng
rng = default_rng()


folder = "../../results_all/app_bootstrap_influx/"
resultfile = "../../results_aggr/app_bootstrap_influx.csv"
print(folder)


all_files = glob.glob(folder + "/*")
rows = []

for file in all_files:
    print(f"Paring file {file}")

    values = pd.read_csv(file)

    for index, row in values.iterrows():
        rows.append({
            "commit":row["commit"],
            "type": row["type"],
            "min": row["min"],
            "med": row["med"],
            "max": row["max"]
        })

print(f"Parsing complete")

../../results_all/app_bootstrap_influx/
Paring file ../../results_all/app_bootstrap_influx\app_bootstrap_influxdb_1.csv
Paring file ../../results_all/app_bootstrap_influx\app_bootstrap_influxdb_10.csv
Paring file ../../results_all/app_bootstrap_influx\app_bootstrap_influxdb_100.csv
Paring file ../../results_all/app_bootstrap_influx\app_bootstrap_influxdb_101.csv
Paring file ../../results_all/app_bootstrap_influx\app_bootstrap_influxdb_102.csv
Paring file ../../results_all/app_bootstrap_influx\app_bootstrap_influxdb_103.csv
Paring file ../../results_all/app_bootstrap_influx\app_bootstrap_influxdb_104.csv
Paring file ../../results_all/app_bootstrap_influx\app_bootstrap_influxdb_105.csv
Paring file ../../results_all/app_bootstrap_influx\app_bootstrap_influxdb_106.csv
Paring file ../../results_all/app_bootstrap_influx\app_bootstrap_influxdb_107.csv
Paring file ../../results_all/app_bootstrap_influx\app_bootstrap_influxdb_108.csv
Paring file ../../results_all/app_bootstrap_influx\app_bootst

In [4]:
print(f"Saving to {resultfile}")
df_result = pd.DataFrame(rows)
# Save to file
df_result.to_csv(str(resultfile))
print("Done.")




Saving to ../../results_aggr/app_bootstrap_influx.csv
Done.
